In [1]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5000data_training.csv",sep=';', encoding='utf-8')
data.head()

,user,Comment,Label
0,AsepSul46335267,Wong cilik yang mba...,NON-CYBERBULYING
1,pengushrnminari,Aku wong cilik tapi pake sepatu bu,NON-CYBERBULYING
2,Kontol893,Ahahaha. Kontlo. Niatan mau merakyat malah ga...,CYBERBULLYING
3,Hajir11336579,Mantaab mbak puan..maju trus..smoga jadi pres...,NON-CYBERBULYING
4,abon_kang,"Wes tah puan, rekam jejakmu ora genah blas ðŸ‘Ž",CYBERBULLYING


In [4]:
data = data.drop(columns=['user'], axis=1)
data.head()

,Comment,Label
0,Wong cilik yang mba...,NON-CYBERBULYING
1,Aku wong cilik tapi pake sepatu bu,NON-CYBERBULYING
2,Ahahaha. Kontlo. Niatan mau merakyat malah ga...,CYBERBULLYING
3,Mantaab mbak puan..maju trus..smoga jadi pres...,NON-CYBERBULYING
4,"Wes tah puan, rekam jejakmu ora genah blas ðŸ‘Ž",CYBERBULLYING


In [5]:
data.isnull().sum()

Comment    0
Label      0
dtype: int64

In [6]:
import re

def casefolding(comment):
  comment = comment.lower().strip(" ")
  comment = re.sub(r'[?|$|.|!²_:")(-+,]','', comment)
  return comment
  
data['Comment'] = data['Comment'].apply(casefolding)
data.head()

,Comment,Label
0,wong cilik yang mba,NON-CYBERBULYING
1,aku wong cilik tapi pake sepatu bu,NON-CYBERBULYING
2,ahahaha kontlo niatan mau merakyat malah gak n...,CYBERBULLYING
3,mantaab mbak puanmaju trussmoga jadi presiden,NON-CYBERBULYING
4,wes tah puan rekam jejakmu ora genah blas ðÿ‘ž,CYBERBULLYING


In [7]:
def token(comments):
  nstr = comments.split(' ')
  dat = []
  a = -1
  for hu in nstr:
    a += 1
  if hu == '':
    dat.append(a)
  p = b = 0
  for q in dat:
      b = q - p
      del nstr[b]
      p += 1
  return nstr
data['Comment'] = data['Comment'].apply(token)
data.head(100)

,Comment,Label
0,"[wong, cilik, yang, mba]",NON-CYBERBULYING
1,"[aku, wong, cilik, tapi, pake, sepatu, bu]",NON-CYBERBULYING
2,"[ahahaha, kontlo, niatan, mau, merakyat, malah...",CYBERBULLYING
3,"[mantaab, mbak, puanmaju, trussmoga, jadi, pre...",NON-CYBERBULYING
4,"[wes, tah, puan, rekam, jejakmu, ora, genah, b...",CYBERBULLYING
...,...,...
95,"[iyain, aja, biar, seneng]",NON-CYBERBULYING
96,"[wong, cilik, apa, wong, licik]",CYBERBULLYING
97,[skip],NON-CYBERBULYING
98,"[taik, kau]",CYBERBULLYING


In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(comments):
  filtering = stopwords.words('indonesian','english')
  x = data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, comments)
  for i in fit:
    data.append(i)
  return data
data['Comment'] = data['Comment'].apply(stopword_removal)
data.head(100)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Comment,Label
0,"[cilik, mba]",NON-CYBERBULYING
1,"[cilik, pake, sepatu, bu]",NON-CYBERBULYING
2,"[ahahaha, kontlo, niatan, merakyat, gak, ngerti]",CYBERBULLYING
3,"[mantaab, mbak, puanmaju, trussmoga, presiden]",NON-CYBERBULYING
4,"[wes, tah, puan, rekam, jejakmu, ora, genah, b...",CYBERBULLYING
...,...,...
95,"[iyain, aja, biar, seneng]",NON-CYBERBULYING
96,"[cilik, licik]",CYBERBULLYING
97,[skip],NON-CYBERBULYING
98,"[taik, kau]",CYBERBULLYING


In [9]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(comments):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in comments:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  return d_clean
data['Comment'] = data['Comment'].apply(stemming)

data.to_csv('data_clean.csv', index=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 21.0 MB/s 


In [10]:
cyberbullying_clean = pd.read_csv('data_clean.csv', encoding='latin1')
cyberbullying_clean.dropna()

,Comment,Label
0,cilik mba,NON-CYBERBULYING
1,cilik pake sepatu bu,NON-CYBERBULYING
2,ahahaha kontlo niat rakyat gak ngerti,CYBERBULLYING
3,mantaab mbak puanmaju trussmoga presiden,NON-CYBERBULYING
4,wes tah puan rekam jejak ora genah blas,CYBERBULLYING
...,...,...
4995,sedng kritik naik eeeh micnya dimatiin bela ci...,CYBERBULLYING
4996,bukti bijak cilik blt yg dl cela era sby,NON-CYBERBULYING
4997,warung mane warungnye,NON-CYBERBULYING
4998,lambe bosok belatung,CYBERBULLYING


In [11]:
cyberbullying_clean = cyberbullying_clean.astype({'Label': 'category'})
cyberbullying_clean = cyberbullying_clean.astype({'Comment': 'string'})

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(cyberbullying_clean['Comment'].astype("U"))
text_tf

<5000x7307 sparse matrix of type '<class 'numpy.float64'>'
	with 28773 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_tf, cyberbullying_clean['Label'], test_size=0.2, random_state = 42)

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, classification_report

clf = SVC()
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
predicted = clf.predict(X_test)

print("Accuracy", accuracy_score(y_test, predicted))
print("Precision", precision_score(y_test, predicted,average="binary", pos_label="CYBERBULLYING"))
print("Recall", recall_score(y_test, predicted,average="binary", pos_label="CYBERBULLYING"))
print("F1 Score", f1_score(y_test, predicted,average="binary", pos_label="CYBERBULLYING"))

print(classification_report(y_test, predicted, zero_division=0))

Accuracy 0.788
Precision 0.6428571428571429
Recall 0.08181818181818182
F1 Score 0.14516129032258066
                  precision    recall  f1-score   support

   CYBERBULLYING       0.64      0.08      0.15       220
NON-CYBERBULYING       0.79      0.99      0.88       780

        accuracy                           0.79      1000
       macro avg       0.72      0.53      0.51      1000
    weighted avg       0.76      0.79      0.72      1000

